In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets, linear_model, preprocessing
from sklearn.metrics import mean_squared_error, r2_score
import sklearn
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [3]:
data = pd.read_csv('kc_house_data.csv');

In [4]:
data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [5]:
## Shuffle the data
data = data.sample(frac=1).reset_index(drop=True)

In [6]:
zip_code_ohe = OneHotEncoder();

In [7]:
x = zip_code_ohe.fit_transform(data.zipcode.values.reshape(-1,1)).toarray()

In [8]:
df_ohe = pd.DataFrame(x, columns=['zipcode_' + str(int(i)) for i in range(x.shape[1])])

In [9]:
data = pd.concat([data, df_ohe], axis=1)

In [10]:
x_data = data.drop(columns=['id', 'price', 'lat', 'long', 'date', 'zipcode'])

In [11]:
y_data = data['price']

In [12]:
## This is a kitchen sink regression

In [13]:
x_data = np.array(x_data)
y_data = np.array(y_data)

In [14]:
train_data = x_data[:-4322]
test_data = x_data[-4322:]

In [15]:
train_label = y_data[:-4322]
test_label = y_data[-4322:]

In [16]:
train_data.shape

(17291, 85)

In [17]:
regr = linear_model.LinearRegression(fit_intercept=False)

In [18]:
regr.fit(train_data, train_label);

In [19]:
train_prediction = regr.predict(train_data)

In [20]:
test_prediction = regr.predict(test_data)

In [21]:
print('Coeff: \n', regr.coef_);

Coeff: 
 [-2.71375130e+04  1.98946334e+04 -5.95167538e+12  2.39697193e-01
 -4.57812527e+04  6.55175525e+05  5.60599807e+04  2.46992054e+04
  5.46498246e+04  5.95167538e+12  5.95167538e+12 -6.80033527e+02
  1.77015278e+01  1.14311759e+01 -1.09940933e-01  8.40858394e+05
  8.72457248e+05  8.26112996e+05  1.62661551e+06  1.14343042e+06
  1.10741663e+06  1.08440006e+06  1.09045091e+06  9.02255800e+05
  9.56991071e+05  9.34447358e+05  9.32489553e+05  8.22754501e+05
  8.08711193e+05  1.00545528e+06  1.00583566e+06  9.63622095e+05
  1.05175770e+06  8.53566703e+05  8.57881581e+05  8.48684186e+05
  1.20794247e+06  1.04047885e+06  8.72179721e+05  2.03274701e+06
  1.34451114e+06  8.46531717e+05  9.30836807e+05  1.07044619e+06
  1.02771758e+06  8.93958982e+05  9.34288973e+05  8.72344490e+05
  9.22062211e+05  9.27257846e+05  8.13541738e+05  9.89119520e+05
  1.01839532e+06  1.00612215e+06  9.55566421e+05  8.13715749e+05
  1.37416558e+06  1.18759986e+06  1.32063133e+06  9.96145721e+05
  1.20147023e+06

In [22]:
mse = mean_squared_error(test_label, test_prediction)
print('MSE ', mse)

MSE  25380886197.824486


In [23]:
import math
print('RMSQ', math.sqrt(mse))

RMSQ 159313.79788902306


In [24]:
print('R2 Value: ', r2_score(test_label, test_prediction));

R2 Value:  0.8248898227179617


In [25]:
## Same model with NN using Keras

In [26]:
data = pd.read_csv('kc_house_data.csv');
data = data.sample(frac=1).reset_index(drop=True)
## Drop all categorical features 
normalized_data = data.drop(columns=['id', 'price', 'lat', 'long', 'date', 'zipcode', 'floors', 'waterfront', 'view', 'condition', 'grade'])

In [27]:
ss = preprocessing.StandardScaler()

In [28]:
normalized_data = ss.fit_transform(normalized_data)

In [29]:
normalized_data[:10]

array([[ 0.67648506, -0.79823546, -0.7076283 , -0.21069681, -0.79508887,
         0.01918579, -0.8853496 , -0.21012839, -0.60777271, -0.23397896],
       [ 0.67648506, -0.14900736, -0.19587967, -0.16772193,  0.13478216,
        -0.65868104,  0.13600588, -0.21012839, -1.03089846, -0.22841191],
       [ 0.67648506,  0.50022075,  0.52274607, -0.26767473,  0.93181448,
        -0.65868104,  1.19140654, -0.21012839,  1.4932655 , -0.28759836],
       [-0.39873715, -0.14900736, -0.62052215, -0.27395195, -1.03661382,
         0.65186149, -1.56625325, -0.21012839, -0.69531597, -0.32993718],
       [-0.39873715, -1.44746357, -1.11049425, -0.0750121 , -0.8796226 ,
        -0.65868104, -0.64703332, -0.21012839, -0.63695379, -0.08308282],
       [-0.39873715, -0.14900736, -0.56608081,  0.41115925, -1.06076631,
         0.81003042,  0.17005106, -0.21012839, -0.75367814,  0.5999786 ],
       [ 0.67648506,  0.8248348 ,  0.60985222, -0.21407685,  1.02842445,
        -0.65868104,  1.02118063, -0.21012839

In [30]:
normalized_data = pd.DataFrame(normalized_data)

In [31]:
floors = data['floors']
waterfront = data['waterfront']
view = data['view']
condition = data['condition']
grade = data['grade']

In [32]:
x_data = pd.concat([normalized_data,floors,waterfront, view, condition, grade ], axis=1)
y_data = data['price']

In [33]:
x = zip_code_ohe.fit_transform(data.zipcode.values.reshape(-1,1)).toarray()

In [34]:
df_ohe = pd.DataFrame(x, columns=['zipcode_' + str(int(i)) for i in range(x.shape[1])])
x_data = pd.concat([x_data, df_ohe],axis=1)

In [35]:
x_data.head()

,0,1,2,3,4,5,6,7,8,9,floors,waterfront,view,condition,grade,zipcode_0,zipcode_1,zipcode_2,zipcode_3,zipcode_4,zipcode_5,zipcode_6,zipcode_7,zipcode_8,zipcode_9,zipcode_10,zipcode_11,zipcode_12,zipcode_13,zipcode_14,zipcode_15,zipcode_16,zipcode_17,zipcode_18,zipcode_19,zipcode_20,zipcode_21,zipcode_22,zipcode_23,zipcode_24,...,zipcode_30,zipcode_31,zipcode_32,zipcode_33,zipcode_34,zipcode_35,zipcode_36,zipcode_37,zipcode_38,zipcode_39,zipcode_40,zipcode_41,zipcode_42,zipcode_43,zipcode_44,zipcode_45,zipcode_46,zipcode_47,zipcode_48,zipcode_49,zipcode_50,zipcode_51,zipcode_52,zipcode_53,zipcode_54,zipcode_55,zipcode_56,zipcode_57,zipcode_58,zipcode_59,zipcode_60,zipcode_61,zipcode_62,zipcode_63,zipcode_64,zipcode_65,zipcode_66,zipcode_67,zipcode_68,zipcode_69
0,0.676485,-0.798235,-0.707628,-0.210697,-0.795089,0.019186,-0.885350,-0.210128,-0.607773,-0.233979,1.5,0,0,4,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.676485,-0.149007,-0.195880,-0.167722,0.134782,-0.658681,0.136006,-0.210128,-1.030898,-0.228412,1.0,0,0,3,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.676485,0.500221,0.522746,-0.267675,0.931814,-0.658681,1.191407,-0.210128,1.493265,-0.287598,2.0,0,0,3,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.398737,-0.149007,-0.620522,-0.273952,-1.036614,0.651861,-1.566253,-0.210128,-0.695316,-0.329937,1.0,0,0,5,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.398737,-1.447464,-1.110494,-0.075012,-0.879623,-0.658681,-0.647033,-0.210128,-0.636954,-0.083083,1.0,0,0,3,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
x_data = np.array(x_data)
y_data = np.array(y_data)

In [37]:
train_data = x_data[:-4322]
test_data = x_data[-4322:]
train_label = y_data[:-4322]
test_label = y_data[-4322:]

In [50]:
def build_model():
  model = keras.Sequential([
                            keras.layers.Dense(64, activation=tf.nn.relu, input_shape=(train_data.shape[1], )),
                            keras.layers.Dense(64, activation=tf.nn.relu), 
                            keras.layers.Dense(1)
                            ])
  optimizer = tf.optimizers.Adam(0.01)
  model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
  return model

In [51]:
model = build_model()
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 64)                5504      
_________________________________________________________________
dense_15 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 65        
Total params: 9,729
Trainable params: 9,729
Non-trainable params: 0
_________________________________________________________________


In [54]:
model.fit(train_data, train_label, validation_split=0.2, epochs=100, verbose=0)

In [55]:
loss,mse = model.evaluate(test_data, test_label)

136/136 [==============================] - 0s 2ms/step - loss: 14984118272.0000 - mae: 72139.5781


In [56]:
print('MSE ', mse)

MSE  72139.578125


In [57]:
print('Loss ', loss)

Loss  14984118272.0


In [58]:
train_predict = model.predict(train_data)

In [59]:
test_predict = model.predict(test_data).flatten()

In [60]:
print("R2 Value of NN ", r2_score(train_label, train_predict))

R2 Value of NN  0.9032295997706536


In [61]:
print("R2 Value of NN ", r2_score(test_label, test_predict))

R2 Value of NN  0.9054246735760723
